In [2]:
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search


train_df = pd.read_csv("E:/IIITH AIML/week12/experiment3/Accident_train.csv")
test_df = pd.read_csv("E:/IIITH AIML/week12/experiment3/Accident_test.csv")
combine = [train_df, test_df]


C:\Users\sravyay\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
week_mapping = {"MON": 1, "TUE": 2, "WED": 3, "THU": 4, "FRI": 5, "SAT": 6, "SUN": 7}
for dataset in combine:
    dataset['Weekday_of_Collision'] = dataset['Weekday_of_Collision'].map(week_mapping)
    dataset['Weekday_of_Collision'] = dataset['Weekday_of_Collision'].fillna(0)

train_df.head()

,Collision_Ref_No,Policing_Area,Collision_Severity,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_HC,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site
0,3518,CREA,3,1,4,8,14.0,13,60,1.0,1.0,1.0,1.0,2,9,9.0,1.0
1,10557,BELC,3,6,8,8,17.0,11,50,12.0,7.0,1.0,1.0,4,3,1.0,1.0
2,5002,LISB,3,3,5,11,17.0,1,60,12.0,7.0,1.0,1.0,2,2,2.0,1.0
3,11714,BELC,3,7,18,10,16.0,12,70,6.0,NaN,1.0,1.0,1,3,1.0,1.0
4,12416,MIDU,3,1,23,11,9.0,13,60,6.0,7.0,1.0,1.0,2,3,1.0,1.0


In [4]:
uniq_pas = train_df["Policing_Area"].unique()
pas_dict = {}
for i in range(0,len(uniq_pas)):
    pas_dict[uniq_pas[i]]=i
    
for dataset in combine:
    dataset['Policing_Area'] = dataset['Policing_Area'].map(pas_dict)
    dataset['Policing_Area'] = dataset['Policing_Area'].fillna(0)

train_df.head()

,Collision_Ref_No,Policing_Area,Collision_Severity,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_HC,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site
0,3518,0,3,1,4,8,14.0,13,60,1.0,1.0,1.0,1.0,2,9,9.0,1.0
1,10557,1,3,6,8,8,17.0,11,50,12.0,7.0,1.0,1.0,4,3,1.0,1.0
2,5002,2,3,3,5,11,17.0,1,60,12.0,7.0,1.0,1.0,2,2,2.0,1.0
3,11714,1,3,7,18,10,16.0,12,70,6.0,NaN,1.0,1.0,1,3,1.0,1.0
4,12416,3,3,1,23,11,9.0,13,60,6.0,7.0,1.0,1.0,2,3,1.0,1.0


In [5]:
for dataset in combine:
    dataset['Policing_Area'] = dataset['Policing_Area'].fillna(train_df.Policing_Area.dropna().mode()[0])
    dataset['Hour_of_Collision'] = dataset['Hour_of_Collision'].fillna(train_df.Hour_of_Collision.dropna().mode()[0])
    dataset['Junction_Detail'] = dataset['Junction_Detail'].fillna(train_df.Junction_Detail.dropna().mode()[0])
    dataset['Junction_Control'] = dataset['Junction_Control'].fillna(train_df.Junction_Control.dropna().mode()[0])
    dataset['Ped_Crossing_HC'] = dataset['Ped_Crossing_HC'].fillna(train_df.Ped_Crossing_HC.dropna().mode()[0])
    dataset['Ped_Crossing_PC'] = dataset['Ped_Crossing_PC'].fillna(train_df.Ped_Crossing_PC.dropna().mode()[0])
    dataset['Road_Surface_Conditions'] = dataset['Road_Surface_Conditions'].fillna(train_df.Road_Surface_Conditions.dropna().mode()[0])
    dataset['Special_Conditions_at_Site'] = dataset['Special_Conditions_at_Site'].fillna(train_df.Special_Conditions_at_Site.dropna().mode()[0])

In [6]:
X_train = train_df.drop(["Collision_Severity"], axis=1)
Y_train = train_df["Collision_Severity"]
X_test  = test_df.drop(["Collision_Severity"], axis=1)
X_train.shape, Y_train.shape, X_test.shape

((10043, 16), (10043L,), (354, 16))

In [7]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [8]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [9]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [25]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True)
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=skf.split(X_train,Y_train),
                   verbose=2, refit=True)

clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    5.8s remaining:    8.7s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   15.6s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   15.6s finished


In [8]:
submission = pd.DataFrame({
        "Collision_Ref_No": 1,
        "Collision_Severity": Y_pred
    })
submission.to_csv('E:/IIITH AIML/week12/experiment3/submission.csv', index=False)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

In [7]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, Y_train)
Y_pred = gbm.predict(X_test)